
---

# 🔹 1. Path Parameters — The Identity System of Your API

---

## 🧠 What is a Path Parameter?

Imagine you're building an **online bookstore API**.

You want users to be able to access:

* All books → `/books`
* A specific book → `/books/42`

Here, `42` is a **path parameter** — it identifies **which** specific book the user wants.

📌 A **path parameter** is:

* A **dynamic value** embedded in the **URL path**
* Used to **target a specific resource** (e.g., user ID, product ID)

---

### ✅ Purpose of Path Parameters:

| Goal            | Explanation                                                             |
| --------------- | ----------------------------------------------------------------------- |
| Identify        | Allows client to specify *which* entity to access (user, product, post) |
| RESTful URLs    | Keeps APIs clean, readable, and consistent                              |
| Dynamic routing | Helps you handle thousands of resources without hardcoding every route  |

---

### 🔧 How Does It Work in FastAPI?

Let’s write a route in FastAPI that fetches a book by its ID:

```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/books/{book_id}")
def get_book(book_id: int):
    return {"book_id": book_id, "title": f"Book #{book_id}"}
```

When a user accesses `/books/10`, FastAPI:

* Extracts `10` from the URL
* Converts it into an `int` (`book_id`)
* Passes it to the function

✨ Magic of **type hints** — if you send `/books/ten`, FastAPI throws a 422 error.

---

## 🏗️ What Can `@app.path()` or `@app.get()` Do?

In FastAPI, `@app.get()`, `@app.post()`, `@app.put()`... all allow **path parameter injection**.

You define parameters like this:

```python
@app.get("/users/{user_id}/orders/{order_id}")
def get_user_order(user_id: int, order_id: int):
    ...
```

It supports:

* **Multiple path parameters**
* **Typed path parameters** (`int`, `str`, `UUID`)
* **Dependency injection** even on path-based functions
* **Auto docs** from the parameters

---

### 📦 Real Case Scenario: E-Commerce Order Tracking

Let’s say a user wants to track a specific order:

```
GET /users/101/orders/909
```

In code:

```python
@app.get("/users/{user_id}/orders/{order_id}")
def track_order(user_id: int, order_id: int):
    return {
        "user": user_id,
        "order": order_id,
        "status": "Shipped"
    }
```

No query strings. No noise. Just clean RESTful path structure.

---

# 🔹 2. HTTP Status Codes — The Emotions of the Web

Every time an API responds, it sends a **status code** — like an emoji expressing how it feels about your request.

---

## 🗂️ Most Important HTTP Status Codes

Let’s break them into categories — with **real-life examples**:

---

### ✅ 200 OK

* Meaning: Success, request processed correctly
* Example: `GET /books/1` returns the book

---

### 🆕 201 Created

* Meaning: Something new was successfully created
* Example: `POST /books` returns a success after inserting a book
* FastAPI: You should return with `status_code=201`

```python
@app.post("/books", status_code=201)
def create_book(book: Book):
    ...
```

---

### ❌ 400 Bad Request

* Meaning: You sent something invalid
* Example: Missing required fields, wrong types
* Happens when: Pydantic validation fails

---

### 🚫 401 Unauthorized

* Meaning: You’re not logged in
* Example: Trying to access `/profile` without logging in
* You’d use this when validating JWT tokens or login state

---

### 🔐 403 Forbidden

* Meaning: You’re logged in but **not allowed** to do this
* Example: User tries to access admin-only data

---

### 🚫 404 Not Found

* Meaning: The resource you’re asking for doesn’t exist
* Example: `GET /books/9999` when no such book exists

---

### ⚠️ 422 Unprocessable Entity

* Meaning: Input was structured properly but semantically wrong
* Common when: Pydantic can’t coerce your input

---

### 💥 500 Internal Server Error

* Meaning: Something broke on the server
* Example: Unhandled exceptions, bad DB queries
* You should **avoid exposing** these errors — log them instead!

---

# 🔹 3. HTTPException — Your Custom Error Thrower

Now, let’s say your app receives:

```
GET /books/9999
```

But your database has no book with ID 9999.
You don’t want to return a 200 with `null` — that’s misleading.
You should raise an **HTTP 404**.

This is where FastAPI’s `HTTPException` comes in.

---

### 🧪 Real Case Example:

```python
from fastapi import HTTPException

@app.get("/books/{book_id}")
def get_book(book_id: int):
    if book_id not in database:
        raise HTTPException(status_code=404, detail="Book not found")
    return database[book_id]
```

This:

* Returns status code 404
* Includes the message `"Book not found"`
* Shows up in Swagger docs

---

### ⚙️ You Can Use It For:

| Use Case                      | Status Code | Detail                |
| ----------------------------- | ----------- | --------------------- |
| Missing resource              | 404         | "Not found"           |
| Bad token                     | 401         | "Invalid credentials" |
| Accessing someone else’s data | 403         | "Access denied"       |
| Validation failure (manual)   | 400         | "Field X required"    |

---

### 🎁 Pro Tip:

You can even customize the response headers:

```python
raise HTTPException(
    status_code=403,
    detail="You don’t own this data",
    headers={"X-App-Error": "AuthFail"}
)
```

---

# ✅ Summary

| Topic           | What It Is                      | Example                      |
| --------------- | ------------------------------- | ---------------------------- |
| Path Param      | Dynamic value in URL            | `/books/45`                  |
| GET             | Retrieve info                   | `@app.get()`                 |
| POST            | Create new data                 | `@app.post(status_code=201)` |
| PUT             | Update existing data            | `@app.put()`                 |
| DELETE          | Remove a resource               | `@app.delete()`              |
| HTTPException   | Custom error handling           | `raise HTTPException(404)`   |
| Status Code 200 | Success                         | View resource                |
| Status Code 401 | Unauthorized                    | No login token               |
| Status Code 403 | Forbidden                       | No permission                |
| Status Code 404 | Not found                       | Book doesn't exist           |
| Status Code 422 | Unprocessable (validation fail) | Wrong input                  |

---
